<a href="https://colab.research.google.com/github/jeosol/ai-api/blob/main/Spam_Classifier_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Spam Classifier with Keras
With deep learning and AI, handling spam content has gotten easier and easier. Over time (and with the aid of direct user feedback) our spam classifier will rarely produce erroneous results. 

This is the first part of a multi-part series covering how to:

- Build an AI Model (this one)
- Integrate a NoSQL Database (inference result storing)
- Deploy an AI Model into Production

### Prerequisites
- Prepare your dataset using [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/1%20-%20Prepare%20the%20AI%20Spam%20Classifier%20Dataset.ipynb) .
- Convert your dataset into trainable vectors in [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb) (Either way, this notebook will run this step for us).


### Running this notebook:
- Recommended: Use [Colab](https://colab.research.google.com/github/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/Spam_Classifier_with_Keras.ipynb) as it offers free GPUs for training models. [Launch this notebook here]([Colab](https://colab.research.google.com/github/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/Spam_Classifier_with_Keras.ipynb))
- Fork [the AI as an API repo](https://github.com/codingforentrepreneurs/AI-as-an-API) and run `guides/spam-classifier/Spam_Classifier_with_Keras.ipynb` whenever you'd like.

This notebook is brought to in you in partnership with [DataStax](https://dtsx.io/3nRWZEG).

In [1]:
#!sudo update-alternatives --config python3
#!python3 --version
#!sudo update-alternatives --set python /usr/bin/python3.9
!pip install boto3
!pip install pandas tensorflow

     |████████████████████████████████| 131 kB 5.3 MB/s 
     |████████████████████████████████| 8.1 MB 41.7 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 138 kB 45.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 127 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: pip's

In [2]:
import boto3
import os
import pathlib
import pandas as pd
import pickle

In [3]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
EXPORT_DIR = pathlib.Path('/datasets/exports/')
GUIDES_DIR = pathlib.Path("/guides/spam-classifier/")
DATASET_CSV_PATH = EXPORT_DIR / 'spam-dataset.csv'
TRAINING_DATA_PATH = EXPORT_DIR / 'spam-training-data.pkl'
PART_TWO_GUIDE_PATH = GUIDES_DIR / "2 - Convert Dataset into Vectors.ipynb"

## Prepare Dataset

Creating a dataset rarely happens next to where you run the training. The below cells are a method for us to extract the needed data to perform training against.

In [5]:
!mkdir -p "$EXPORT_DIR"
!mkdir -p "$GUIDES_DIR"
!curl "https://raw.githubusercontent.com/jeosol/ai-api/main/datasets/exports/spam-dataset.csv" -o "$DATASET_CSV_PATH"
!curl "https://raw.githubusercontent.com/jeosol/ai-api/main/notebooks/5%20-%20Convert%20Dataset%20into%20Vectors,%20Split%20&%20Export.ipynb" -o "$PART_TWO_GUIDE_PATH"
#!curl "https://raw.githubusercontent.com/codingforentrepreneurs/AI-as-an-API/main/datasets/exports/spam-dataset.csv" -o "$DATASET_CSV_PATH"
#!curl "https://raw.githubusercontent.com/codingforentrepreneurs/AI-as-an-API/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb" -o "$PART_TWO_GUIDE_PATH"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  741k  100  741k    0     0  2362k      0 --:--:-- --:--:-- --:--:-- 2362k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  124k  100  124k    0     0   567k      0 --:--:-- --:--:-- --:--:--  565k


Let's review our extracted dataset which combines two different spam datasets as outlined in [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/1%20-%20Prepare%20the%20AI%20Spam%20Classifier%20Dataset.ipynb).

In [6]:
print(PART_TWO_GUIDE_PATH)

/guides/spam-classifier/2 - Convert Dataset into Vectors.ipynb


In [7]:
df = pd.read_csv(DATASET_CSV_PATH, error_bad_lines=False)
df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",sms-spam
1,ham,Ok lar... Joking wif u oni...,sms-spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,sms-spam
3,ham,U dun say so early hor... U c already then say...,sms-spam
4,ham,"Nah I don't think he goes to usf, he lives aro...",sms-spam


In [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb) we prepare our dataset (`spam-dataset.csv`) to be fully ready for training on a model. Below is a command to run that notebook.

In [8]:
%run "$PART_TWO_GUIDE_PATH"

Extract prepared training dataset results.

In [9]:
data = {}

with open(TRAINING_DATA_PATH, 'rb') as f:
    data = pickle.load(f)

In [10]:
data

{'X_test': array([[  0,   0,   0, ...,  11,  70,  19],
        [  0,   0,   0, ...,   7, 165,  25],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ..., 206,  56,   5],
        [  0,   0,   0, ...,  16,  73,  19],
        [  0,   0,   0, ...,  26, 104, 106]], dtype=int32),
 'X_train': array([[  0,   0,   0, ...,  77, 263, 218],
        [  0,   0,   0, ..., 261,  12,  18],
        [  0,   0,   0, ...,  12,  46, 245],
        ...,
        [  0,   0,   0, ...,   0,   0,   1],
        [  0,   0,   0, ...,  30, 182,   9],
        [  0,   0,   0, ...,   0,   3, 156]], dtype=int32),
 'labels_legend_inverted': {'0': 'ham', '1': 'spam'},
 'legend': {'ham': 0, 'spam': 1},
 'max_sequence': 280,
 'max_words': 280,
 'tokenizer': <keras_preprocessing.text.Tokenizer at 0x7f7e304e0b10>,
 'y_test': array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=float32),
 'y_train': array([[1., 0.],
       

> While the above code uses `pickle` to load in data, this data is actually exported via `pickle` when we execute the `%run` only a few steps ago. Since `pickle` can be unsafe to use from third-party downloaded data, we actually generate (again using `%run`) this pickle data and therefore is safe to use -- it's never downloaded.

## Transform Extracted Dataset

In [11]:
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']
labels_legend_inverted = data['labels_legend_inverted']
legend = data['legend']
max_sequence = data['max_sequence']
max_words = data['max_words']
tokenizer = data['tokenizer']

## Create our LSTM Model

In [12]:
embed_dim = 128
lstm_out = 196
# Need to understand the neural network model here; Add a discussion in a cell above for the architecture
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, embed_dim, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3)) # LSTM used for text related data, dropout and recurrent dropout (study these)
model.add(Dense(2, activation='softmax')) # specify the number of classes here, get a softmax corresponding to probabilities of each class
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 280, 128)          35840     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 280, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 291,034
Trainable params: 291,034
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# split the data to do validation and testing later on. Current format uses the test data as validation
batch_size = 32
epochs = 5
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, verbose=1, epochs=epochs)

Epoch 1/5
158/158 [==============================] - 329s 2s/step - loss: 0.2862 - accuracy: 0.8868 - val_loss: 0.1391 - val_accuracy: 0.9509
Epoch 2/5
158/158 [==============================] - 318s 2s/step - loss: 0.1403 - accuracy: 0.9552 - val_loss: 0.1381 - val_accuracy: 0.9561
Epoch 3/5
158/158 [==============================] - 323s 2s/step - loss: 0.1340 - accuracy: 0.9554 - val_loss: 0.1287 - val_accuracy: 0.9565
Epoch 4/5
158/158 [==============================] - 319s 2s/step - loss: 0.1252 - accuracy: 0.9593 - val_loss: 0.1391 - val_accuracy: 0.9553
Epoch 5/5
158/158 [==============================] - 322s 2s/step - loss: 0.1199 - accuracy: 0.9627 - val_loss: 0.1261 - val_accuracy: 0.9610


In [14]:
MODEL_EXPORT_PATH = EXPORT_DIR / 'spam-model.h5'
model.save(str(MODEL_EXPORT_PATH))

In [54]:
model

## Predict new data

In [55]:
import numpy as np

def predict(text_str, max_words=280, max_sequence = 280, tokenizer=None):
  if not tokenizer:
    return None
  sequences = tokenizer.texts_to_sequences([text_str])
  x_input = pad_sequences(sequences, maxlen=max_sequence)
  y_output = model.predict(x_input)
  top_y_index = np.argmax(y_output)
  preds = y_output[top_y_index]      
  preds = model.predict(x_input)
  labeled_preds = [{f"{labels_legend_inverted[str(i)]}": x} for i, x in enumerate(preds)]
  return labeled_preds

In [56]:
predict("Hello world", max_words=max_words, max_sequence=max_sequence, tokenizer=tokenizer)

[{'ham': array([0.94839007, 0.05160988], dtype=float32)}]

## Exporting Tokenizer & Metadata

In [ ]:
import json
metadata = {
    "labels_legend_inverted": labels_legend_inverted,
    "legend": legend,
    "max_sequence": max_sequence,
    "max_words": max_words,
}

METADATA_EXPORT_PATH = EXPORT_DIR / 'spam-classifer-metadata.json'
METADATA_EXPORT_PATH.write_text(json.dumps(metadata, indent=4))

187

In [ ]:
tokenizer_as_json = tokenizer.to_json()

TOKENIZER_EXPORT_PATH = EXPORT_DIR / 'spam-classifer-tokenizer.json'
TOKENIZER_EXPORT_PATH.write_text(tokenizer_as_json)

827612

We can load `tokenizer_as_json` with `tensorflow.keras.preprocessing.text.tokenizer_from_json`.

## Upload Model, Tokenizer, & Metadata to Object Storage

Object Storage options include:

- AWS S3
- Linode Object Storage
- DigitalOcean Spaces


All three of these options can use `boto3`.

In [ ]:
# AWS S3 Config
ACCESS_KEY = "<your_aws_iam_key_id>"
SECRET_KEY = "<your_aws_iam_secret_key>"

# You should not have to set this
ENDPOINT = None

# Your s3-bucket region
REGION = 'us-west-1'

BUCKET_NAME = '<your_s3_bucket_name>'

#### Linode Object Storage Config

In [ ]:
ACCESS_KEY = "<your_linode_object_storage_access_key>"
SECRET_KEY = "<your_linode_object_storage_secret_key>"

# Object Storage Endpoint URL
ENDPOINT = "https://cfe3.us-east-1.linodeobjects.com"

# Object Storage Endpoint Region (also in your endpoint url)
REGION = 'us-east-1'

# Set this to a valid slug (without a "/" )
BUCKET_NAME = 'datasets'

#### DigitalOcean Spaces Config

In [ ]:
ACCESS_KEY = "<your_do_spaces_access_key>"
SECRET_KEY = "<your_do_spaces_secret_key>"

# Space Endpoint URL
ENDPOINT = "https://ai-cfe-1.nyc3.digitaloceanspaces.com"

# Space Region (also in your endpoint url)
REGION = 'nyc3'

# Set this to a valid slug (without a "/" )
BUCKET_NAME = 'datasets'

## Perform Upload with Boto3

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = SECRET_KEY

In [ ]:
# Upload paths 
MODEL_KEY_NAME = f"exports/spam-sms/{MODEL_EXPORT_PATH.name}"
TOKENIZER_KEY_NAME = f"exports/spam-sms/{TOKENIZER_EXPORT_PATH.name}"
METADATA_KEY_NAME = f"exports/spam-sms/{METADATA_EXPORT_PATH.name}"

In [ ]:
session = boto3.session.Session()
client = session.client('s3', region_name=REGION, endpoint_url=ENDPOINT)
client.upload_file(str(MODEL_EXPORT_PATH), BUCKET_NAME,  MODEL_KEY_NAME) 
client.upload_file(str(TOKENIZER_EXPORT_PATH), BUCKET_NAME,  TOKENIZER_KEY_NAME) 
client.upload_file(str(METADATA_EXPORT_PATH), BUCKET_NAME,  METADATA_KEY_NAME)  

In [ ]:
client.download_file(BUCKET_NAME, MODEL_KEY_NAME, pathlib.Path(MODEL_KEY_NAME).name)
client.download_file(BUCKET_NAME, TOKENIZER_KEY_NAME, pathlib.Path(TOKENIZER_KEY_NAME).name)
client.download_file(BUCKET_NAME, METADATA_KEY_NAME, pathlib.Path(METADATA_KEY_NAME).name)

## Implement an AI Model Download Pipeline
In [this blog post](https://www.codingforentrepreneurs.com/blog/ai-model-download-pipeline) I'll show you how to turn the `client.download_file()` portion into a pipeline so you can make it reusable in future projects. Further, if you ever need to bundle these models into a Docker image, you will be able to use the pipeline.